# MOSP

In a minimization of open stacks problem (MOSP), we are given a set of customers $C = \{ 0, ..., n-1 \}$ and a set of products $P = \{ 0, ..., m-1 \}$.
Each customer $c \in C$ requests a subset of products $P_c \subseteq P$.
We want to decide the order to produce the products.
Once we produced a product $p \in P_c$, we need to open a stack for customer $c$ to store the product.
When we produced all products in $P_c$, we can close the stack for $c$.
We want to minimize the maximum number of open stacks at a time.

## DP Formulation

The approach is to find the order of customers to close stacks instead of the order of products to produce.
Once the order of customers is determined, for each customer, products requested by the customer that are not produced yet are consecutively produced in an arbitrary order.

When we close the stack for customer $c$, we need to produce all products in $P_c$.
If another customer $c'$ requestss a product in $P_c$ and its stack is not opened yet, we need to open the stack for $c'$.
In a sense, we can say that $c'$ is a neighbor of $c$.
Let $N_c \subseteq C$ be the set of neighbors including $c$, i.e., $N_c = \{ c' \in C \mid P_{c'} \cap P_c \neq \emptyset \}$.

Let $O$ be the set of customers whose stacks have been opened.
When we are producing the products requested by $c$, we need to open new stacks for customers $N_c \setminus O$.
Let $R$ be the set of customers whose stacks are not closed yet.
Because the set of customers whose stacks have been opened and not closed is $O \cap R$, the number of open stacks when producing the products for $c$ is $|(O \cap R) \cup (N_c \setminus O)|$.

When we close the stack for $c$, the set of customers whose stacks are not closed becomes $R \setminus \{ c \}$, and the set of customers whose stacks have been opened becomes $O \cup N_c$.
Let $V(R, O)$ be the minimum of the maximum number of open stacks at a time to close the stacks for customers in $R$ when the stacks for customers in $O$ have been opened.
Then, the DP formulation is

$$
\begin{align}
    \text{compute } & V(C, \emptyset) \\
    & V(R, O) = \begin{cases}
        \min\limits_{c \in R} \max\left\{ |(O \cap R) \cup (N_c \setminus O)|,  V(R \setminus \{ c \}, O \cup N_c) \right\} & \text{if } R \neq \emptyset \\
        0 & \text{if } R = \emptyset
    \end{cases} \\
    & V(R, O) \geq 0.
\end{align}
$$

## Install DIDPPy

In [1]:
!pip install didppy

## Data

In [2]:
# Number of customers
n = 4
# Number of items
m = 4
# Items requested by customers
customer_to_items = [{0}, {0, 1}, {2}, {1}]

## Preprocessiong

In [3]:
# Neighbors
neighbors = [[] for _ in range(n)]

for i in range(n):
    for j in range(n):
        if len(customer_to_items[i] & customer_to_items[j]) > 0:
            neighbors[i].append(j)

## Modeling

In [4]:
import didppy as dp

model = dp.Model()

customer = model.add_object_type(number=n)

# R
remaining = model.add_set_var(object_type=customer, target=list(range(n)))
# O
opened = model.add_set_var(object_type=customer, target=[])

neighbor_table = model.add_set_table(neighbors, object_type=customer)

for c in range(n):
    close = dp.Transition(
        name="close {}".format(c),
        cost=dp.max(
            ((opened & remaining) | (neighbor_table[c] - opened)).len(),
            dp.IntExpr.state_cost(),
        ),
        effects=[
            (remaining, remaining.remove(c)),
            (opened, opened | neighbor_table[c]),
        ],
        preconditions=[remaining.contains(c)],
    )
    model.add_transition(close)

model.add_base_case([remaining.is_empty()])

model.add_dual_bound(0)

## Solving

In [5]:
solver = dp.CABS(model, f_operator=dp.FOperator.Max, quiet=True)
solution = solver.search()

print("Transitions to apply:")
print("")

for t in solution.transitions:
    print(t.name)

print("")
print("Cost: {}".format(solution.cost))

Transitions to apply:

close 2
close 0
close 1
close 3

Cost: 2
